# Exploratory Analysis of Restaurants

# Business Questions
* How are outpost locations selected? Particularly given outposts are nearly ~50% of ordering locations
* On Presidents Day 93 locations were closed for ordering. How does that affect demand? How to restaurants change the amount they order to accomodate? What about expired inventory?
* What is Sweetgreen Brink Demo? Why is meat disabled?

In [ ]:
import sweetgreen as sg
import pandas as pd
import matplotlib.pyplot as plt
import copy

In [2]:
restaurants = sg.utils.read_json("../data/cleaned/flattened_restaurants.json")
df = pd.DataFrame(restaurants)
df.head()

,Fri_end,Fri_start,Mon_end,Mon_start,Sat_end,Sat_start,Sun_end,Sun_start,Thu_end,Thu_start,...,show_warning_dialog,state,store_hours,supports_mixing,throttle_default_size,throttle_enabled,warning_dialog_description,warning_dialog_timeout,warning_dialog_title,zip_code
0,22:00,10:30,22:00,10:30,22:00,10:30,22:00,10:30,22:00,10:30,...,False,DC,Mon - Sun 10:30am - 10pm,True,35,True,None,-1,None,20036
1,22:00,10:30,22:00,10:30,22:00,10:30,22:00,10:30,22:00,10:30,...,False,MD,Mon - Sun 10:30am - 10pm,True,40,True,None,-1,None,20814
2,22:00,10:30,22:00,10:30,22:00,10:30,22:00,10:30,22:00,10:30,...,False,DC,Mon - Sun 10:30am - 10pm,True,50,True,None,-1,None,20003
3,22:00,10:30,22:00,10:30,21:00,10:30,21:00,10:30,22:00,10:30,...,False,VA,Mon - Fri 10:30am - 10pm \r\nSat - Sun 10:30am...,True,30,True,None,-1,None,20190
4,22:00,10:30,22:00,10:30,22:00,10:30,22:00,10:30,22:00,10:30,...,False,VA,Mon - Sun 10:30am - 10pm,True,23,True,None,-1,None,22203


In [3]:
df.iloc[0]

Fri_end                                                                     22:00
Fri_start                                                                   10:30
Mon_end                                                                     22:00
Mon_start                                                                   10:30
Sat_end                                                                     22:00
Sat_start                                                                   10:30
Sun_end                                                                     22:00
Sun_start                                                                   10:30
Thu_end                                                                     22:00
Thu_start                                                                   10:30
Tue_end                                                                     22:00
Tue_start                                                                   10:30
Wed_end         

# Summary numbers for restaurants

In [4]:
df.shape

(207, 50)

In [5]:
df["accepting_orders"].value_counts()

True     204
False      3
Name: accepting_orders, dtype: int64

In [6]:
df["is_outpost"].value_counts()

False    108
True      99
Name: is_outpost, dtype: int64

In [7]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df[~df["enabled"]])


,Fri_end,Fri_start,Mon_end,Mon_start,Sat_end,Sat_start,Sun_end,Sun_start,Thu_end,Thu_start,Tue_end,Tue_start,Wed_end,Wed_start,accepting_orders,address,asset_ids,available_dropoff_locations,bag_product_id,city,cross_street,dietary_preference_overrides,distance,enabled,entity_id,flex_message,hidden,id,is_outpost,latitude,longitude,menu_id,name,not_accepting_orders_reason,olo_id,phone,pickup_instructions,restaurant_slug,show_mixing_tooltip,show_upsells,show_warning_dialog,state,store_hours,supports_mixing,throttle_default_size,throttle_enabled,warning_dialog_description,warning_dialog_timeout,warning_dialog_title,zip_code
25,22:00,10:30,22:00,10:30,22:00,10:30,22:00,10:30,22:00,10:30,22:00,10:30,22:00,10:30,True,3333 M St NW,[1336],[],None,Washington,None,[],None,False,NaN,,True,26,False,38.905311,-77.067335,26,Georgetown Old,Online ordering is currently unavailable. We a...,13881,12023379338,,georgetown-old,False,True,False,DC,Mon - Sun 10:30am - 10pm,False,23,False,None,-1,None,20007
41,22:00,10:30,22:00,10:30,22:00,10:30,22:00,10:30,22:00,10:30,22:00,10:30,22:00,10:30,False,1890 Shattuck Ave,[1507],[],None,Berkeley,None,[],None,False,43.0,,False,42,False,37.873993,-122.268823,42,Berkeley,This location does not currently offer online ...,25774,15109908262,Head to the online ordering pickup area straig...,berkeley,False,True,False,CA,Mon - Sun 10:30am - 10pm,False,23,False,None,-1,None,94709


In [8]:
df["enabled"].value_counts()

True     205
False      2
Name: enabled, dtype: int64

In [9]:
df["show_upsells"].value_counts()

False    105
True     102
Name: show_upsells, dtype: int64

In [10]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df[~df["accepting_orders"]])

,Fri_end,Fri_start,Mon_end,Mon_start,Sat_end,Sat_start,Sun_end,Sun_start,Thu_end,Thu_start,Tue_end,Tue_start,Wed_end,Wed_start,accepting_orders,address,asset_ids,available_dropoff_locations,bag_product_id,city,cross_street,dietary_preference_overrides,distance,enabled,entity_id,flex_message,hidden,id,is_outpost,latitude,longitude,menu_id,name,not_accepting_orders_reason,olo_id,phone,pickup_instructions,restaurant_slug,show_mixing_tooltip,show_upsells,show_warning_dialog,state,store_hours,supports_mixing,throttle_default_size,throttle_enabled,warning_dialog_description,warning_dialog_timeout,warning_dialog_title,zip_code
41,22:00,10:30,22:00,10:30,22:00,10:30,22:00,10:30,22:00,10:30,22:00,10:30,22:00,10:30,False,1890 Shattuck Ave,[1507],[],None,Berkeley,None,[],None,False,43.0,,False,42,False,37.873993,-122.268823,42,Berkeley,This location does not currently offer online ...,25774,15109908262,Head to the online ordering pickup area straig...,berkeley,False,True,False,CA,Mon - Sun 10:30am - 10pm,False,23,False,None,-1,None,94709
110,12:45,12:15,12:45,12:15,None,None,None,None,12:45,12:15,12:45,12:15,12:45,12:15,False,11677 San Vicente Blvd,[439],[],None,Los Angeles,None,[],None,True,10067.0,,True,111,False,34.054171,-118.465136,110,Headspace-Old,All Sweetgreen Outposts will be closed from 12...,66338,11111111111,,headspace-old,False,False,False,CA,,False,23,False,None,-1,None,90049
125,12:45,12:15,None,None,None,None,None,None,12:45,12:15,12:45,12:15,12:45,12:15,False,12130 Millennium Drive,[6197],[],None,Los Angeles,None,[],None,True,60082.0,Honest Company employees only for this sweetgr...,False,126,True,33.979982,-118.406361,126,sg Outpost at Honest Company,This sg Outpost is closed in observation of Pr...,71461,13103406880,Head to the kitchen on the 4th floor at 12:30p...,honest-company,False,False,True,CA,,False,45,True,At this time only Honest Company employees can...,-1,Are you an Honest Company employee?,90094


In [14]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df[df["dietary_preference_overrides"] != "[]"])

,Fri_end,Fri_start,Mon_end,Mon_start,Sat_end,Sat_start,Sun_end,Sun_start,Thu_end,Thu_start,Tue_end,Tue_start,Wed_end,Wed_start,accepting_orders,address,asset_ids,available_dropoff_locations,bag_product_id,city,cross_street,dietary_preference_overrides,distance,enabled,entity_id,flex_message,hidden,id,is_outpost,latitude,longitude,menu_id,name,not_accepting_orders_reason,olo_id,phone,pickup_instructions,restaurant_slug,show_mixing_tooltip,show_upsells,show_warning_dialog,state,store_hours,supports_mixing,throttle_default_size,throttle_enabled,warning_dialog_description,warning_dialog_timeout,warning_dialog_title,zip_code
31,22:00,11:00,22:00,11:00,22:00,11:00,22:00,11:00,22:00,11:00,22:00,08:00,23:30,11:00,True,26 Broadway,[439],[],None,New York,None,['meat'],None,True,100000001.0,,True,32,False,40.705442,-74.012888,32,Sweetgreen Brink Demo,Online ordering is currently unavailable. We a...,17260,15555555555,Head to your assigned shelf where bowls are al...,sweetgreen-brink-demo,True,True,False,NY,,True,200,True,None,-1,None,10004


# Load Presidents day JSON and run same analysis

In [11]:
restaurants_pres_day = sg.utils.read_json("../data/cleaned/flattened_restaurants_presidents_day.json")
df_pres_day= pd.DataFrame(restaurants_pres_day)

In [12]:
df_pres_day["accepting_orders"].value_counts()

True     112
False     95
Name: accepting_orders, dtype: int64